# Assess predictions on object detection fridge data with a pytorch model

This notebook demonstrates the use of the `responsibleai` API to assess an object detection pytorch model trained on the fridge dataset. It walks through the API calls necessary to create a widget with model analysis insights, then guides a visual analysis of the model.

## Launch Responsible AI Toolbox

The following section examines the code necessary to create datasets and a model. It then generates insights using the `responsibleai` API that can be visually analyzed.

### Load Model and Data

In [ ]:
import os
import sys
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from responsibleai_vision.common.constants import ImageColumns
import json
from raiutils.common.retries import retry_function

import json
import os
import xml.etree.ElementTree as ET

try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

EPOCHS = 10
LEARNING_RATE = 1e-4
IM_SIZE = 300
BATCH_SIZE = 16
FRIDGE_MODEL_NAME = 'fridge_od_model'
FRIDGE_MODEL_WINDOWS_NAME = 'fridge_model_windows'
WIN = 'win'

def load_fridge_object_detection_dataset_labels():

    src_images = "./data/odFridgeObjects/"

    # Path to the annotations
    annotations_folder = os.path.join(src_images, "annotations")
    
    labels = []
    label_dict = {'can': 1, 'carton': 2, 'milk_bottle': 3, 'water_bottle': 4}
    
    # Read each annotation
    for i, filename in enumerate(os.listdir(annotations_folder)):
        if filename.endswith(".xml"):
            print("Parsing " + os.path.join(src_images, filename))

            root = ET.parse(os.path.join(annotations_folder, filename)).getroot()

            # use if needed
            # width = int(root.find("size/width").text)
            # height = int(root.find("size/height").text)

            image_labels = []
            for object in root.findall("object"):
                name = object.find("name").text
                xmin = object.find("bndbox/xmin").text
                ymin = object.find("bndbox/ymin").text
                xmax = object.find("bndbox/xmax").text
                ymax = object.find("bndbox/ymax").text
                isCrowd = int(object.find("difficult").text)
                image_labels.append([
                    label_dict[name],  # label
                    float(xmin),  # topX. To normalize, divide by width.
                    float(ymin),  # topY. To normalize, divide by height.
                    float(xmax),  # bottomX. To normalize, divide by width
                    float(ymax),  # bottomY. To normalize, divide by height
                    int(isCrowd)
                ])
            labels.append(image_labels)
    
    return labels

def load_fridge_object_detection_dataset():
    # create data folder if it doesnt exist.
    os.makedirs("data", exist_ok=True)

    # download data
    download_url = ("https://cvbp-secondary.z19.web.core.windows.net/" +
                    "datasets/object_detection/odFridgeObjects.zip")
    data_file = "./odFridgeObjects.zip"
    urlretrieve(download_url, filename=data_file)

    # extract files
    with ZipFile(data_file, "r") as zip:
        print("extracting files...")
        zip.extractall(path="./data")
        print("done")
    # delete zip file
    os.remove(data_file)
        
        
    labels = load_fridge_object_detection_dataset_labels()
    
    # get all file names into a pandas dataframe with the labels
    data = pd.DataFrame(columns=[ImageColumns.IMAGE.value,
                                    ImageColumns.LABEL.value])
    for i, file in enumerate(os.listdir("./data/odFridgeObjects/" + "images")):
        image_path = "./data/odFridgeObjects/" + "images" + "/" + file
        data = data.append({ImageColumns.IMAGE.value: image_path,
                            ImageColumns.LABEL.value: labels[i]}, # folder
                            ignore_index=True)
    
    
    return data

### Load Pretrained Faster RCNN model

In [ ]:
import urllib.request as request_file

#download fine-tuned recycling model from url
def download_assets(filepath,force=False):
    if force or not os.path.exists(filepath):
        request_file.urlretrieve(
                        "https://publictestdatasets.blob.core.windows.net/models/fastrcnn.pt",
                        os.path.join(filepath))
    else:
        print('Found' + filepath)

    return filepath

In [ ]:
#Loading in our pretrained model     
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
import os

def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Device: ', str(device))

num_classes = 5
model = get_instance_segmentation_model(num_classes)
_ = download_assets('Recycling_finetuned_FastRCNN.pt')
model.load_state_dict(torch.load('Recycling_finetuned_FastRCNN.pt', map_location = device))

#if using the general torchvision pretrained model, comment above and uncomment below
# model = detection.fasterrcnn_resnet50_fpn(pretrained=True)

model.to(device)

In [ ]:
data = load_fridge_object_detection_dataset()

train_data = data
test_data = data
class_names = np.array(['can', 'carton', 'milk_bottle', 'water_bottle'])

### Create Model and Data Insights

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai_vision import ModelTask, RAIVisionInsights

To use Responsible AI Toolbox, initialize a RAIInsights object upon which different components can be loaded.

RAIInsights accepts the model, the full dataset, the test dataset, the target feature string and the task type string as its arguments.

In [ ]:
rai_insights = RAIVisionInsights(model, test_data.sample(5, random_state=42),
                                 "label",
                                 task_type=ModelTask.OBJECT_DETECTION,
                                 classes=class_names)

rai_insights

Add the components of the toolbox that are focused on model assessment.

In [ ]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis - to be added after support in future
# rai_insights.error_analysis.add()

Once all the desired components have been loaded, compute insights on the test set.

In [ ]:
rai_insights.compute()

Finally, visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab.

In [ ]:
ResponsibleAIDashboard(rai_insights)